# Importing Libraries and Defining Parameters
- In this section, we import the necessary libraries and define the initial parameters, such as image size, batch size, and the data directory.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 256, 256
batch_size = 32
data_dir = "../cherry-leaves"


2025-03-08 05:24:31.819922: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Configuring ImageDataGenerator
- We configure the ImageDataGenerator to perform data augmentation and split the data into training and validation sets.

In [2]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% training, 20% validation
)

### Creating Data Generators for Training and Validation
- Here, we create the generators that will read images directly from the directory, applying preprocessing and splitting them into training and validation sets.

In [3]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification: healthy vs. powdery mildew
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Found 3368 images belonging to 2 classes.
Found 840 images belonging to 2 classes.
